In [1]:
import autograd.numpy as np

from autograd import grad, jacobian

from scipy.linalg import sqrtm

In [2]:
import SREKF_Core

In [3]:
#radius of the Earth in meters
R_EARTH = 6.3781363e6
OMEGA_EARTH = 7.292115146706979e-5

In [4]:
#simulation timestep used
h=1

In [5]:
#Exponential fit model from SMAD data
def atm_density(alt):
    
    atm_d = np.exp(-1.63481928e-2*alt) * np.exp(-2.00838711e1)
    
    return atm_d

In [6]:
def measurement_function(x): 
    
    C = np.hstack((np.eye(3), np.zeros((3,9))))
    
    #only return the position 
    measurement = C@x
    
    return measurement, C

In [8]:
#checked with julia dynamics and they match
def process_dynamics(x, h):
    #position 
    q = x[0:3]
    v = x[3:6]
    a_d = x[6:9]
    beta = x[9:12]
    
    #drag coefficient
    cd = 2.0 
    
    #cross sectional area
    A= 0.1
    
    omega_earth = np.array([0,0, OMEGA_EARTH])
    
    v_rel = v - np.cross(omega_earth, q)
    
    #get the altititude in km
    alt = np.linalg.norm(q) - (R_EARTH*1e-3)
    
    #estimated rho
    rho_est = atm_density(alt)
    
    #drag force
    f_drag = -0.5*cd*(A)*rho_est*np.linalg.norm(v_rel)*v_rel
    
    μ = 3.986004418e14 #m3/s2
    J2 = 1.08264e-3 
    
    a_2bp = (-μ*q)/(np.linalg.norm(q))**3
    
    Iz = np.array([0,0,1])
    
    a_J2 = ((3*μ*J2*R_EARTH**2)/(2*np.linalg.norm(q)**5))*((((5*np.dot(q, Iz)**2)/np.linalg.norm(q)**2)-1)*q - 2*np.dot(q,Iz)*Iz)   
    
    a = a_2bp + a_J2 + f_drag + a_d
    
    a_d_dot = -np.diag(beta)@a_d
    
    x_dot = np.concatenate([v, a, a_d_dot, np.zeros(3)])
        
    return x_dot

In [9]:
def RK4_process_dynamics(x, h): 
    
    f1= process_dynamics(x, h) 
    f2 = process_dynamics(x+0.5*h*f1, h)
    f3 = process_dynamics(x+0.5*h*f2, h)
    f4 = process_dynamics(x+h*f3, h)
    
    xnext = x+(h/6.0)*(f1+2*f2+2*f3+f4)
    
    return xnext

In [10]:
a = np.array([1,2,3])
b = np.array([2,3,4])

c = np.hstack((a,b))

In [11]:
std_gps_measurement = 10
std_velocity = 0.01

In [12]:
#original
gps_noise = np.random.normal(size=3)*10/3
velocity_noise = np.random.normal(size=3)*0.001 

In [14]:
#Initialize the SREKF Core Class 

#true first pose plus some noise

initial_state = np.array([-6.291285386249863e6+gps_noise[0], -1.66264810311861e6+gps_noise[1], -2.2161566541800327e6 + gps_noise[2], -2394.9708335284654 + velocity_noise[0], -540.1532693380549 + velocity_noise[1], 7206.151917789134 + velocity_noise[2], 0, 0, 0, 1e-3, 1e-3, 1e-3])
initial_P = np.identity(12)*np.hstack((np.ones(3)*((std_gps_measurement)**2)/3, np.ones(3)*((std_velocity)**2)/3, np.ones(3)*5e-4**2, np.ones(3)*4e-4**2))
initial_F = sqrtm(initial_P)

In [15]:
#tuning parameters for the 
q_betas = 2e-9*np.ones(3) 
q_eps= 5.5e-11*np.ones(3)

In [16]:
R_measure = np.identity(3)*((std_gps_measurement)**2)/3

In [17]:
ekf = SREKF_Core.EKFCore(initial_state, initial_F, RK4_process_dynamics, measurement_function, q_betas, q_eps, R_measure)

In [18]:
h = 1 #using a timestep of 1

In [19]:
#first measurement
y = np.array([  -6.29366686958334e6,
 -1.6631941155051156e6,
 -2.208956584174462e6])

In [20]:
#the measurement should be of the next time step (k+1)
ekf.update(y, h)

this is y:  [-6293666.86958334 -1663194.11550512 -2208956.58417446]
this is y_predicted [-6293675.18937362 -1663191.61591346 -2208947.93965697]
this is z:  [ 8.31979028 -2.49959166 -8.64451749]
